<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/DINOV3_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install --upgrade transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 768.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s 

Apply for the model access: https://huggingface.co/facebook/dinov3-vits16-pretrain-lvd1689m

In [11]:
from huggingface_hub import login
from huggingface_hub import whoami

YOUR_HF_TOKEN = ''
# Replace 'YOUR_HF_TOKEN' with your Hugging Face access token
login(token=YOUR_HF_TOKEN)

info = whoami()
print("Logged in as:", info["name"])

Logged in as: Mobarakol


In [14]:
# pip install torchao
import torch
from transformers import TorchAoConfig, AutoImageProcessor, AutoModel
from torchao.quantization import Int4WeightOnlyConfig
from transformers.image_utils import load_image


url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = load_image(url)

processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vitsplus-pretrain-lvd1689m")

quant_type = Int4WeightOnlyConfig(group_size=128)
quantization_config = TorchAoConfig(quant_type=quant_type)

model_name = "facebook/dinov3_vits14"

model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=quantization_config
)

inputs = processor(images=image, return_tensors="pt").to(model.device)
with torch.inference_mode():
    outputs = model(**inputs)

pooled_output = outputs.pooler_output
print("Pooled output shape:", pooled_output.shape)


OSError: facebook/dinov3-vitsplus-pretrain-lvd1689m is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [8]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import torch
import requests

# 1. Load the model and processor
# We are using 'facebook/dinov3-vits14', a small and fast DINOv3 version.
processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vits14")
model = AutoModel.from_pretrained("facebook/dinov3-vits14")

# 2. Prepare your image
# You can load an image from a local path or a URL.
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# 3. Process the image and get inputs for the model
inputs = processor(images=image, return_tensors="pt")

# 4. Perform a forward pass to get the features
# We use torch.no_grad() to disable gradient calculations for faster inference.
with torch.no_grad():
    outputs = model(**inputs)

# The last hidden state contains the patch features and the CLS token.
last_hidden_state = outputs.last_hidden_state

# 5. Extract the desired features
# The CLS token (first token) is often used as a global image representation.
cls_token_embedding = last_hidden_state[:, 0]

# The remaining tokens are the patch features, which represent different parts of the image.
patch_embeddings = last_hidden_state[:, 1:]

# Print the shapes of the extracted features
print("Shape of the CLS token embedding:", cls_token_embedding.shape)
print("Shape of the patch embeddings:", patch_embeddings.shape)

OSError: facebook/dinov3-vits14 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [3]:
from PIL import Image
import torch
from transformers import AutoFeatureExtractor, AutoModel

# Load the model and feature extractor
model_name = "facebookresearch/dinov3-small"
model = AutoModel.from_pretrained(model_name)
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)

# Load the image
image = Image.open("path_to_your_image.jpg")

# Preprocess the image
inputs = feature_extractor(images=image, return_tensors="pt")

# Extract features
with torch.no_grad():
    outputs = model(**inputs)
    features = outputs.last_hidden_state[:, 0, :]  # Use the CLS token representation

# Print the features
print(features.shape)
print(features)

OSError: facebookresearch/dinov3-small is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`